<a href="https://colab.research.google.com/github/ckenlam/NLU-Prepositions-Challenge/blob/main/Challenge_Problem_Preposition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 3.5 MB 17.8 MB/s 
     |████████████████████████████████| 895 kB 28.9 MB/s 
     |████████████████████████████████| 67 kB 2.9 MB/s 
     |████████████████████████████████| 6.8 MB 39.0 MB/s 
     |████████████████████████████████| 596 kB 41.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [14]:
!pip install datasets

     |████████████████████████████████| 311 kB 24.9 MB/s 
     |████████████████████████████████| 1.1 MB 42.9 MB/s 
     |████████████████████████████████| 243 kB 46.3 MB/s 
     |████████████████████████████████| 133 kB 49.5 MB/s 
     |████████████████████████████████| 144 kB 44.2 MB/s 
     |████████████████████████████████| 271 kB 46.6 MB/s 
     |████████████████████████████████| 94 kB 3.1 MB/s 


In [3]:
!pip install transformers[sentencepiece]

     |████████████████████████████████| 1.2 MB 19.9 MB/s 


In [2]:
import transformers

In [4]:
from transformers import TFAutoModelForMaskedLM

model_checkpoint = "distilbert-base-uncased"
model = TFAutoModelForMaskedLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/347M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForMaskedLM: ['activation_13']
- This IS expected if you are initializing TFDistilBertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertForMaskedLM were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForMaskedLM for predictions without further training.


In [5]:
model(model.dummy_inputs)  # Build the model
model.summary()

Model: "tf_distil_bert_for_masked_lm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMai  multiple                 66362880  
 nLayer)                                                         
                                                                 
 vocab_transform (Dense)     multiple                  590592    
                                                                 
 vocab_layer_norm (LayerNorm  multiple                 1536      
 alization)                                                      
                                                                 
 vocab_projector (TFDistilBe  multiple                 23866170  
 rtLMHead)                                                       
                                                                 
Total params: 66,985,530
Trainable params: 66,985,530
Non-trainable params: 0
__________________________

In [10]:
text = "This is a great [MASK]."

In [11]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [12]:
import numpy as np
import tensorflow as tf

inputs = tokenizer(text, return_tensors="np")
token_logits = model(**inputs).logits
# Find the location of [MASK] and extract its logits
mask_token_index = np.argwhere(inputs["input_ids"] == tokenizer.mask_token_id)[0, 1]
mask_token_logits = token_logits[0, mask_token_index, :]
# Pick the [MASK] candidates with the highest logits
# We negate the array before argsort to get the largest, not the smallest, logits
top_5_tokens = np.argsort(-mask_token_logits)[:5].tolist()

for token in top_5_tokens:
    print(f">>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}")

>>> This is a great deal.
>>> This is a great success.
>>> This is a great adventure.
>>> This is a great idea.
>>> This is a great feat.


In [187]:
from datasets import load_dataset
dataset = load_dataset('text', data_files={'train': 'https://raw.githubusercontent.com/ckenlam/Language-Model/main/hound-train.txt', 'test':'https://www.gutenberg.org/cache/epub/2852/pg2852.txt'})
dataset

Using custom data configuration default-3ad063869b1cab2a
Reusing dataset text (/root/.cache/huggingface/datasets/text/default-3ad063869b1cab2a/0.0.0/08f6fb1dd2dab0a18ea441c359e1d63794ea8cb53e7863e6edf8fc5655e47ec4)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 12310
    })
    test: Dataset({
        features: ['text'],
        num_rows: 7222
    })
})

In [200]:
dataset['train'][:20]

{'text': ['',
  "Project Gutenberg's The Adventures of Sherlock Holmes, by Arthur Conan Doyle",
  '',
  'This eBook is for the use of anyone anywhere at no cost and with',
  'almost no restrictions whatsoever.  You may copy it, give it away or',
  're-use it under the terms of the Project Gutenberg License included',
  'with this eBook or online at www.gutenberg.net',
  '',
  '',
  'Title: The Adventures of Sherlock Holmes',
  '',
  'Author: Arthur Conan Doyle',
  '',
  'Release Date: November 29, 2002 [EBook #1661]',
  'Last Updated: May 20, 2019',
  '',
  'Language: English',
  '',
  'Character set encoding: UTF-8',
  '']}

In [257]:
def masking_prepositions(example):
  to_be_remove = {'***'}
  example['text']= ' '.join('' if i in to_be_remove else i for i in example['text'].lower().split())
  prepositions = {'of','of','in', 'to', 'for', 'with', 'on', 'at', 'from', 'by', 'about'}
  example['mask']= ' '.join(tokenizer.mask_token if i in prepositions else i for i in example['text'].split())
  return example


def tokenize_function(examples):
    result = tokenizer(examples["mask"])
    result_label = tokenizer(examples["text"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    result['labels'] = result_label['input_ids']
    return result



In [258]:
dataset_masked = dataset.map(masking_prepositions)

tokenized_datasets = dataset_masked.map(tokenize_function,batched=True, remove_columns=["mask","text"])

0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

  0%|          | 0/13 [00:00<?, ?ba/s]

  0%|          | 0/8 [00:00<?, ?ba/s]

In [260]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 12310
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 7222
    })
})

In [261]:
tokenizer.decode(tokenized_datasets["train"][20]["input_ids"])


'[CLS] start [MASK] this project gutenberg ebook the adventures [MASK] sherlock holmes [SEP]'

In [262]:
chunk_size = 128

In [263]:
def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # Compute length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size
    # Split by chunks of max_len
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # Create a new labels column
    # result["labels"] = result["input_ids"].copy()
    return result

In [264]:
lm_datasets = tokenized_datasets.map(group_texts, batched=True)
lm_datasets

  0%|          | 0/13 [00:00<?, ?ba/s]

  0%|          | 0/8 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 1265
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 721
    })
})

In [265]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

'2019 [SEP] [CLS] [SEP] [CLS] language : english [SEP] [CLS] [SEP] [CLS] character set encoding : utf - 8 [SEP] [CLS] [SEP] [CLS] start [MASK] this project gutenberg ebook the adventures [MASK] sherlock holmes [SEP] [CLS] [SEP] [CLS] [SEP] [CLS] [SEP] [CLS] produced [MASK] an anonymous project gutenberg volunteer and jose menendez [SEP] [CLS] [SEP] [CLS] [SEP] [CLS] [SEP] [CLS] cover [SEP] [CLS] [SEP] [CLS] [SEP] [CLS] [SEP] [CLS] the adventures [MASK] sherlock holmes [SEP] [CLS] [SEP] [CLS] [SEP] [CLS] [SEP] [CLS] [MASK] arthur conan doyle [SEP] [CLS] [SEP] [CLS] [SEP] [CLS] [SEP] [CLS] contents [SEP] [CLS] [SEP] [CLS] [SEP] [CLS] i. a scandal [MASK] bohemia [SEP] [CLS] ii. the red - headed league [SEP] [CLS] iii. a case [MASK] identity'